In [3]:
import numpy as np
#from netCDF4 import Dataset as ncdset
import glob
import warnings
import datetime
#import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from mpl_toolkits.basemap import Basemap
import os
#import cdsapi
#from scipy.optimize import curve_fit
import xarray as xr
import xskillscore as xs
import pandas as pd
import xesmf as xesmf

#### Puts all observational datasets on common 1deg x 1deg grid
#### and standardizes variable names, units, and conventions

In [4]:
def regrid_with_nan(data,regridder,C=10.):
    data['pr'] = data['pr'] + C
    data['tas'] = data['tas'] + C
    data_rg = regridder(data)
    data_rg['pr'] = xr.where(data_rg['pr']==0.0,np.nan,data_rg['pr'])
    data_rg['tas'] = xr.where(data_rg['tas']==0.0,np.nan,data_rg['tas'])
    data_rg['pr'] = data_rg['pr'] - C
    data_rg['tas'] = data_rg['tas'] - C
    data['pr'] = data['pr'] - C
    data['tas'] = data['tas'] - C
    return data_rg

In [5]:
diri = '/glade/work/nlybarger/data/OBS/'

In [6]:
gmet = xr.open_dataset(diri + 'GMET/gmet.1970-2021.e1-10.monthly.nc',drop_variables=['t_range'])
livneh = xr.open_dataset(diri + 'Livneh/livneh_prec.mon.mean.nc')
livneh = xr.merge([livneh, xr.open_dataset(diri + 'Livneh/livneh_trad.mon.mean.nc')])
cru = xr.open_dataset(diri + 'CRU/cru_ts4.06.1901.2021.pre.dat.nc',drop_variables=['stn'])
cru = xr.merge([cru, xr.open_dataset(diri + 'CRU/cru_ts4.06.1901.2021.tmp.dat.nc',drop_variables=['stn'])])
era5 = xr.open_dataset(diri + 'ERA-5/era5.mon.T.P.global_fixed.nc')
udel = xr.open_dataset(diri + 'UDel/precip.mon.total.v501.nc')
udel = xr.merge([udel, xr.open_dataset(diri + 'UDel/air.mon.mean.v501.nc')])
prism = xr.open_dataset(diri + 'PRISM/prism.mon_tot_pr.tas.nc')


In [7]:
cruout = cru
cruout['lon'] = cruout['lon']+360.
cruout = cruout.sel(lat=latslic,lon=lonslic,drop=True)
cruout = cruout.rename_vars({'pre':'pr','tmp':'tas'})
cruout.to_netcdf(diri + 'CRU/fixed.cru.wconus.mon_tot_pr.tas.nc',mode='w')

In [9]:
era5out = era5
month_length = era5.time.dt.days_in_month
era5out['pr'] = era5['pr']*month_length
era5out.to_netcdf(diri + 'ERA-5/fixed.era5.global.mon_tot_pr.tas.nc',mode='w')

In [10]:
udelout = udel.rename_vars({'precip':'pr','air':'tas'})
udelout = udelout.reindex(lat=list(reversed(udel.lat)))
udelout = udelout.sel(lat=latslic,lon=lonslic,drop=True)
udelout['pr'] = udelout['pr']*10
udelout['pr'].attrs['units'] = 'mm'
udelout.to_netcdf(diri + 'UDel/fixed.udel.global.mon_tot_pr.tas.nc',mode='w')

In [23]:
dummy1deg = xr.Dataset(
        data_vars = dict(
    ),
    coords = dict(
        lon = (['lon'], np.arange(1,361)),
        lat = (['lat'], np.arange(-90,91)),
    ),
)



reg = xesmf.Regridder(cruout,dummy1deg,method='bilinear')
cru_r = regrid_with_nan(cruout,reg)
filo = diri + 'CRU/1deg.cru.global.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
cru_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(era5out,dummy1deg,method='bilinear')
era5_r = regrid_with_nan(era5out,reg)
filo = diri + 'ERA-5/1deg.era5.global.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
for it in range(len(era5_r['time'])):
    era5_r['tas'][it,:,:] = xr.where(~np.isnan(cru_r['tas'][0,:,:]),era5_r['tas'][it,:,:],np.nan)
    era5_r['pr'][it,:,:] = xr.where(~np.isnan(cru_r['pr'][0,:,:]),era5_r['pr'][it,:,:],np.nan)
era5_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(udelout,dummy1deg,method='bilinear')
udel_r = regrid_with_nan(udelout,reg)
filo = diri + 'UDel/1deg.udel.global.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
for it in range(len(udel_r['time'])):
    udel_r['tas'][it,:,:] = xr.where(~np.isnan(cru_r['tas'][0,:,:]),udel_r['tas'][it,:,:],np.nan)
    udel_r['pr'][it,:,:] = xr.where(~np.isnan(cru_r['pr'][0,:,:]),udel_r['pr'][it,:,:],np.nan)
udel_r.to_netcdf(filo,mode='w')


/glade/u/home/nlybarger/miniconda3/envs/mypy/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
